#AutoEncoders

##Downloading the dataset

###ML-100K

In [1]:
#!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
   creating: __MACOSX/
   creating: __MACOSX/ml-100k/
  inflating: __MACOSX/ml-100k/._allbut.pl  
  inflating: ml-100k/mku.sh          
  inflating: __MACOSX/ml-100k/._mku.sh  
  inflating: ml-100k/README          
  inflating: __MACOSX/ml-100k/._README  
  inflating: ml-100k/u.data          
  inflating: __MACOSX/ml-100k/._u.data  
  inflating: ml-100k/u.genre         
  inflating: __MACOSX/ml-100k/._u.genre  
  inflating: ml-100k/u.info          
  inflating: __MACOSX/ml-100k/._u.info  
  inflating: ml-100k/u.item          
  inflating: __MACOSX/ml-100k/._u.item  
  inflating: ml-100k/u.occupation    
  inflating: __MACOSX/ml-100k/._u.occupation  
  inflating: ml-100k/u.user          
  inflating: __MACOSX/ml-100k/._u.user  
  inflating: ml-100k/u1.base         
  inflating: __MACOSX/ml-100k/._u1.base  
  inflating: ml-100k/u1.test         
  inflating: __MACOSX/ml-100k/._u1.test  
  inflating: ml-100k/

###ML-1M

In [2]:
#!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/.DS_Store         
   creating: __MACOSX/ml-1m/
  inflating: __MACOSX/ml-1m/._.DS_Store  
  inflating: ml-1m/.Rhistory         
  inflating: ml-1m/movies.dat        
  inflating: __MACOSX/ml-1m/._movies.dat  
  inflating: ml-1m/ratings.dat       
  inflating: __MACOSX/ml-1m/._ratings.dat  
  inflating: ml-1m/README            
  inflating: __MACOSX/ml-1m/._README  
  inflating: ml-1m/test_set.csv      
  inflating: __MACOSX/ml-1m/._test_set.csv  
  inflating: ml-1m/training_set.csv  
  inflating: __MACOSX/ml-1m/._training_set.csv  
  inflating: ml-1m/users.dat         
  inflating: __MACOSX/ml-1m/._users.dat  
  inflating: __MACOSX/._ml-1m        
AutoEncoders                       ~$ Competitive Analysis.docx
AutoEncoders.ipynb                 ~$CT4000-Activity-D02(1).docx
__MACOSX                           ~$CT4003-Case-Project.docx
ml-100k                            ~$CT4008-Activity-06_Solution.docx
ml-100k.zip             

##Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


* sep='::' specifies the separator used to separate values in the file. In this case, it's a double colon '::'. This separator is used because the dataset may not be a standard CSV (Comma-Separated Values) file.

* header=None indicates that there is no header row in the dataset, and the first row should be treated as data, not column names.

* engine='python' specifies the parsing engine to use. In this case, it's set to 'python', which is used when non-standard separators like '::' are used.

* encoding='latin-1' specifies the character encoding used in the file.
'latin-1' is a commonly used encoding for handling text data. (in our case we used latin-1 as data contains special characters or symbols that are better represented in 'latin-1', using this encoding ensures that those characters are correctly read.)



In [4]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [5]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')
# 1 colom - users, 2nd colom - movies and 3rd colom - ratings we dont need 4th colom same in test_set
# in test set we have same users but movies associated with the same user is difffernt from training set

## Getting the number of users and movies


In [6]:
# in upcoming setps we will convert training and test set in to matrix where
# line represents - users, colorings - movies and cells - ratings
# in each of this two matices we will include all users and movies from original dataset

# in cells where user did not rate we will put '0' by default
# we want to get maximum number of users in data set if we get max form training set it wont work
# as users and movies are not distributeed in same way, it possible that user with highest userID could be in test set
# splits are random so we could have highest userID in one of the test set
# we will take max of user_id in training and test sets by doing that we will get max number of user and movies

# we need to do this step if we want to apply 5 k-cross validation sets from u1 to u5

In [7]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
# here we are taking max of both for the user colunm which is 0th colunm same for movies
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns

* [data[:, 0] == id_user] creates a boolean mask by comparing the values in the first column (index 0) of the data array to a specific value id_user. This mask will have True where the condition is met and False where it's not.

In [8]:
# we will create matrice which has max user count as row and max movie count and column

def convert(data):
  # we are creating list of list each line representing unique user
  new_data =[]

  # user value start form 1 to value of nb_user and in range it does not inclue upperbound so we have to add +1
  for id_user in range(1, nb_users + 1):
    id_movies = data[:, 1][data[:, 0] == id_user] # contains index/id of movie rated by each user
    id_ratings = data[:, 2][data[:, 0] == id_user] # contains ratings rated by each user we will put '0' for movies which are not rated

    ratings = np.zeros(nb_movies) # creating rating var containing list of 0, len of total movies

    # index of movies start at 1 but index of rating start at zero so we need to substract by 1
    ratings[id_movies - 1] = id_ratings # now we have list of rating for all the movies ids for all users

    new_data.append(list(ratings)) # added rating to new_data userwise we are creating list of list and rating is array so we need to convert

  return new_data


training_set = convert(training_set)
test_set = convert(test_set)
# so now we have matrices comntaing ratings for all movies for all user( row-user, column each movie)

## Converting the data into Torch tensors


In [9]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [10]:
from torch._C import wait
from torch.nn.modules.loss import MSELoss
# we are creating stacked autoencoder
# In PyTorch, nn.Module is a base class that serves as the foundation for creating neural network models.

class SAE(nn.Module):

  def __init__(self, ):

    super(SAE, self).__init__()
    # By calling super(SAE, self).__init__(), you ensure that the initialization of the nn.Module is performed.

    self.fc1 = nn.Linear(nb_movies, 20)
    # first input is number of features, second input of number of neurons in hidden layer

    self.fc2 = nn.Linear(20, 10)
    # here we are creating another hidden layer which takes 20 node from first hidden layer and 2nd layer has 10 nodes

    self.fc3 = nn.Linear(10, 20)
    # here now we are trying decoding in this layer

    self.fc4 = nn.Linear(20, nb_movies)
    # last decoding layer which takes 20 hidden  node, and number of neurons in output layer(same as input layer)

    self.activation = nn.Sigmoid()


  def forward(self, x):
    # x in put vactor ( input value of input layer )

    x =  self.activation(self.fc1(x))
    # first encoding layer and updating x to pass into next layer

    x =  self.activation(self.fc2(x))
    # second encoding layer and updating x to pass into next layer

    x =  self.activation(self.fc3(x))
    # first decoding layer and updating x to pass into next layer

    x =  self.fc4(x)
    # last decoding layer and updating x to get output, no need of activation function as we are trying to get output layer which has similar value as input layer

    return x


sae = SAE()
criterian = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay=0.5)

## Training the SAE


In [12]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0) # unsqueeze ex.. before [1,2,3] after [[1,2,3]]
    target = input.clone() # creating copy of input

    # to save memory we will filter user if user has not rated even a single movie then we wont perform below operation for that user
    if torch.sum(target.data > 0) > 0:

      output = sae(input) # predicted rating

      target.requires_grad = False
      # to not calculate gradient with respect to target

      output[target == 0] = 0

      loss = criterian(output, target)
      mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
      # mean_corrector: This variable stores the result of the calculation, which can be thought of as a correction factor. It's used to scale the loss
      # This is a small constant (1e-10) added to avoid division by zero.
      # avg or error for movies which are rated

      loss.backward()
      # to know which direction need to update weights

      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.0

      optimizer.step()
      # decides intensity of weight that need update

  print('epoch: ' + str(epoch) + 'loss: ' + str(train_loss/s))


epoch: 1loss: tensor(1.7664)
epoch: 2loss: tensor(1.0965)
epoch: 3loss: tensor(1.0532)
epoch: 4loss: tensor(1.0386)
epoch: 5loss: tensor(1.0308)
epoch: 6loss: tensor(1.0268)
epoch: 7loss: tensor(1.0237)
epoch: 8loss: tensor(1.0218)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0196)
epoch: 11loss: tensor(1.0188)
epoch: 12loss: tensor(1.0184)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0175)
epoch: 15loss: tensor(1.0172)
epoch: 16loss: tensor(1.0171)
epoch: 17loss: tensor(1.0167)
epoch: 18loss: tensor(1.0164)
epoch: 19loss: tensor(1.0164)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0161)
epoch: 22loss: tensor(1.0159)
epoch: 23loss: tensor(1.0158)
epoch: 24loss: tensor(1.0158)
epoch: 25loss: tensor(1.0157)
epoch: 26loss: tensor(1.0156)
epoch: 27loss: tensor(1.0152)
epoch: 28loss: tensor(1.0149)
epoch: 29loss: tensor(1.0126)
epoch: 30loss: tensor(1.0119)
epoch: 31loss: tensor(1.0097)
epoch: 32loss: tensor(1.0091)
epoch: 33loss: tensor(1.0057)
epoch: 34loss: tens

## Testing the SAE


In [15]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0) # unsqueeze ex.. before [1,2,3] after [[1,2,3]]
  # in testing we keep input to training_set as we need to predict based on training data that user liked other movies or not

  target = Variable(test_set[id_user]).unsqueeze(0)
  # taking test set to compare prediction

  # to save memory we will filter user if user has not rated even a single movie then we wont perform below operation for that user
  if torch.sum(target.data > 0) > 0:

    output = sae(input) # predicted rating

    target.requires_grad = False
    # to not calculate gradient with respect to target

    output[target == 0] = 0 # to only consider non-zero ratings

    loss = criterian(output, target)
    mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
    # mean_corrector: This variable stores the result of the calculation, which can be thought of as a correction factor. It's used to scale the loss
    # This is a small constant (1e-10) added to avoid division by zero.
    # avg or error for movies which are rated

    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.0


print('loss: ' + str(test_loss/s))

loss: tensor(0.9605)
